In [1]:
print("Ok")

Ok


In [2]:
!python -V

Python 3.10.14


In [3]:
!pip freeze | grep langchain 

langchain==0.0.225
langchainplus-sdk==0.0.20


### Import required libraries, modules and packages

In [4]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import pinecone
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
import pinecone


In [5]:
PINECONE_API_KEY = "17b0a41d-683a-4974-87ca-a5d2d17ba659"
#PINECONE_API_ENV = "

In [ ]:
# Extracting the text from the pdf file
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob_pattern="*.pdf",
                    loader_cls=PyMuPDFLoader)
    
    documents = loader.load()
    
    return documents